In [1]:
import numpy as np
import pandas as pd

In [2]:
!unzip dataset.csv.zip

Archive:  dataset.csv.zip
replace dataset.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

^C


# Fetch

In [3]:
dataset = pd.read_csv("dataset.csv")
dataset.head()

,Text,Language,Label
0,@Charlie_Corley @Kristine1G @amyklobuchar @Sty...,en,litigious
1,#BadBunny: Como dos gotas de agua: Joven se di...,es,negative
2,https://t.co/YJNiO0p1JV Flagstar Bank disclose...,en,litigious
3,Rwanda is set to host the headquarters of Unit...,en,positive
4,OOPS. I typed her name incorrectly (today’s br...,en,litigious


In [4]:
dataset.describe()

,Text,Language,Label
count,937854,937831,937854
unique,929544,72,4
top,@FlylightsTech Good project,en,positive
freq,73,871310,264545


In [5]:
print(dataset.isnull().sum())

Text         0
Language    23
Label        0
dtype: int64


In [6]:
dataset['Language'].unique()

array(['en', 'es', 'ca', 'fr', 'pt', 'ja', 'hi', 'ar', 'ko', 'nl', 'et',
       'in', 'und', 'tl', 'zh', 'de', 'cs', 'ru', 'qme', 'it', 'el', 'ro',
       'no', 'tr', 'sv', 'ta', 'fa', 'ht', 'pl', 'da', 'th', 'hu', 'lv',
       'uk', 'qht', 'eu', 'qam', 'si', 'cy', 'zxx', 'ml', 'ne', 'mr',
       'qst', 'vi', 'bn', 'gu', 'is', 'fi', 'ckb', nan, 'te', 'art', 'bg',
       'ur', 'sl', 'lt', 'pa', 'iw', 'kn', 'sr',
       "[Photo(previewUrl='https://pbs.twimg.com/media/FWV94O7UEAAjMOi?format=jpg&name=small', fullUrl='https://pbs.twimg.com/media/FWV94O7UEAAjMOi?format=jpg&name=large')]",
       "[Photo(previewUrl='https://pbs.twimg.com/media/FWWBzWTXkAAyZqm?format=jpg&name=small', fullUrl='https://pbs.twimg.com/media/FWWBzWTXkAAyZqm?format=jpg&name=large')]",
       'am', 'or',
       "[Photo(previewUrl='https://pbs.twimg.com/media/EPYG2rKVAAA1e_O?format=jpg&name=small', fullUrl='https://pbs.twimg.com/media/EPYG2rKVAAA1e_O?format=jpg&name=large')]",
       'sd',
       "[Photo(previewUrl='h

In [7]:
dataset['Language'].fillna('', inplace=True)

# Identificar as linhas em que o texto é uma foto
mask_photo = dataset['Language'].astype(str).str.startswith("[Photo")

# Remover as linhas em que o texto é uma foto
dataset = dataset[~mask_photo]


# Certificar-se de lidar com valores NaN adequadamente se houver
dataset.dropna(subset=['Language'], inplace=True)

# Verificar se as linhas foram removidas
print(dataset['Language'].unique())





['en' 'es' 'ca' 'fr' 'pt' 'ja' 'hi' 'ar' 'ko' 'nl' 'et' 'in' 'und' 'tl'
 'zh' 'de' 'cs' 'ru' 'qme' 'it' 'el' 'ro' 'no' 'tr' 'sv' 'ta' 'fa' 'ht'
 'pl' 'da' 'th' 'hu' 'lv' 'uk' 'qht' 'eu' 'qam' 'si' 'cy' 'zxx' 'ml' 'ne'
 'mr' 'qst' 'vi' 'bn' 'gu' 'is' 'fi' 'ckb' '' 'te' 'art' 'bg' 'ur' 'sl'
 'lt' 'pa' 'iw' 'kn' 'sr' 'am' 'or' 'sd'
 "[Video(thumbnailUrl='https://pbs.twimg.com/ext_tw_video_thumb/1541709480324259840/pu/img/E_bkLKnbgX96ui7e.jpg', variants=[VideoVariant(contentType='application/x-mpegURL', url='https://video.twimg.com/ext_tw_video/1541709480324259840/pu/pl/_agIGuySTtX4_1HY.m3u8?tag=12&container=fmp4', bitrate=None), VideoVariant(contentType='video/mp4', url='https://video.twimg.com/ext_tw_video/1541709480324259840/pu/vid/1280x720/VESWY4_w8FIbtAih.mp4?tag=12', bitrate=2176000), VideoVariant(contentType='video/mp4', url='https://video.twimg.com/ext_tw_video/1541709480324259840/pu/vid/480x270/N8UYgrlJKVHmxOkv.mp4?tag=12', bitrate=256000), VideoVariant(contentType='video/mp4', ur

In [8]:
# Identificar as linhas em que o texto é uma foto
mask_photo = dataset['Language'].astype(str).str.startswith("[Video")

# Remover as linhas em que o texto é uma foto
dataset = dataset[~mask_photo]

# Certificar-se de lidar com valores NaN adequadamente se houver
dataset.dropna(subset=['Language'], inplace=True)

# Verificar se as linhas foram removidas
print(dataset['Language'].unique())

['en' 'es' 'ca' 'fr' 'pt' 'ja' 'hi' 'ar' 'ko' 'nl' 'et' 'in' 'und' 'tl'
 'zh' 'de' 'cs' 'ru' 'qme' 'it' 'el' 'ro' 'no' 'tr' 'sv' 'ta' 'fa' 'ht'
 'pl' 'da' 'th' 'hu' 'lv' 'uk' 'qht' 'eu' 'qam' 'si' 'cy' 'zxx' 'ml' 'ne'
 'mr' 'qst' 'vi' 'bn' 'gu' 'is' 'fi' 'ckb' '' 'te' 'art' 'bg' 'ur' 'sl'
 'lt' 'pa' 'iw' 'kn' 'sr' 'am' 'or' 'sd']


In [9]:
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [10]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('all')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/codespace/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     /home/codespace/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     /home/codespace/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /home/codespace/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]   

True

In [11]:
def clean_text(text):
    # Convert to lowercase
    text = str(text).lower()

    # Remove square brackets and contents inside
    text = re.sub(r'\[.*?\]', '', text)

    # Remove URLs
    text = re.sub(r'https?://\S+|www\.\S+', '', text)

    # Remove HTML tags
    text = re.sub(r'<.*?>+', '', text)

    # Remove punctuation
    text = re.sub(rf'[{re.escape(string.punctuation)}]', '', text)

    # Remove newline characters
    text = re.sub(r'\n', '', text)

    # Remove words containing digits
    text = re.sub(r'\w*\d\w*', '', text)

    return text

In [12]:
dataset['Text'] = dataset['Text'].apply(lambda x:clean_text(x))

dataset.head()

,Text,Language,Label
0,charliecorley amyklobuchar stylewriternyc tes...,en,litigious
1,badbunny como dos gotas de agua joven se disfr...,es,negative
2,flagstar bank discloses a data breach that im...,en,litigious
3,rwanda is set to host the headquarters of unit...,en,positive
4,oops i typed her name incorrectly today’s brav...,en,litigious


In [13]:
dataset['Language'] = dataset['Language'].apply(lambda x:clean_text(x))

dataset.head()

,Text,Language,Label
0,charliecorley amyklobuchar stylewriternyc tes...,en,litigious
1,badbunny como dos gotas de agua joven se disfr...,es,negative
2,flagstar bank discloses a data breach that im...,en,litigious
3,rwanda is set to host the headquarters of unit...,en,positive
4,oops i typed her name incorrectly today’s brav...,en,litigious


In [14]:
dataset['Language'].unique()

array(['en', 'es', 'ca', 'fr', 'pt', 'ja', 'hi', 'ar', 'ko', 'nl', 'et',
       'in', 'und', 'tl', 'zh', 'de', 'cs', 'ru', 'qme', 'it', 'el', 'ro',
       'no', 'tr', 'sv', 'ta', 'fa', 'ht', 'pl', 'da', 'th', 'hu', 'lv',
       'uk', 'qht', 'eu', 'qam', 'si', 'cy', 'zxx', 'ml', 'ne', 'mr',
       'qst', 'vi', 'bn', 'gu', 'is', 'fi', 'ckb', '', 'te', 'art', 'bg',
       'ur', 'sl', 'lt', 'pa', 'iw', 'kn', 'sr', 'am', 'or', 'sd'],
      dtype=object)

In [15]:
def remove_stopwords(text):
    if pd.notnull(text):
        stop_words = set(stopwords.words('english'))
        tokens = word_tokenize(text)
        filtered_tokens = [word for word in tokens if word not in stop_words]
        return ' '.join(filtered_tokens)
    return text
def lemmatize_text(text):
    if pd.notnull(text):
        lemmatizer = WordNetLemmatizer()
        tokens = word_tokenize(text)
        lemmatized_text = [lemmatizer.lemmatize(word) for word in tokens]
        return ' '.join(lemmatized_text)
    return text

In [16]:
# Rest of your code
dataset['Text'] = dataset['Text'].apply(lambda x: remove_stopwords(x))


dataset.head()

,Text,Language,Label
0,charliecorley amyklobuchar stylewriternyc test...,en,litigious
1,badbunny como dos gotas de agua joven se disfr...,es,negative
2,flagstar bank discloses data breach impacted i...,en,litigious
3,rwanda set host headquarters united nations de...,en,positive
4,oops typed name incorrectly today ’ brave witn...,en,litigious


In [17]:
dataset['Text'] = dataset['Text'].apply(lambda x: lemmatize_text(x))
dataset.head()

,Text,Language,Label
0,charliecorley amyklobuchar stylewriternyc test...,en,litigious
1,badbunny como do gotas de agua joven se disfra...,es,negative
2,flagstar bank discloses data breach impacted i...,en,litigious
3,rwanda set host headquarters united nation dev...,en,positive
4,oops typed name incorrectly today ’ brave witn...,en,litigious


In [18]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report,accuracy_score
import mlflow
from sklearn.naive_bayes import MultinomialNB


In [19]:
# separar
tweets = dataset['Text']
language_labels = dataset['Language']
sentiment_labels = dataset['Label']


In [20]:
def data_train(mlflow_experiment_id):

    tweets_train, tweets_test, lang_labels_train, lang_labels_test, sent_labels_train, sent_labels_test = train_test_split(
            tweets, language_labels, sentiment_labels, test_size=0.2, random_state=42
        )
   

    with mlflow.start_run(experiment_id=mlflow_experiment_id):
        # classificacao de language
        language_pipeline = Pipeline([
            ('tfidf', TfidfVectorizer()),
            ('clf', LinearSVC())
        ])
        # Train the language  classification model
        language_pipeline.fit(tweets_train, lang_labels_train)


        lang_predictions = language_pipeline.predict(tweets_test)

        # Avaliando o desempenho
        lang_accuracy = accuracy_score(lang_labels_test, lang_predictions)
        lang_report = classification_report(lang_labels_test, lang_predictions)

        mlflow.log_metric("lang_accuracy", lang_accuracy)
        mlflow.log_metric("lang_report", lang_report)


        
        # Define the pipeline for sentiment classification
        sentiment_pipeline = Pipeline([
            ('tfidf', TfidfVectorizer()),
            ('clf', MultinomialNB())
        ])
        # Train the sentiment classification model
        sentiment_pipeline.fit(tweets_train, sent_labels_train)

        # Avaliar o modelo de identificação de sentimento
        sent_predictions = sentiment_pipeline.predict(tweets_test)

        sent_accuracy = accuracy_score(lang_labels_test, sent_predictions)
        sent_report = classification_report(sent_labels_test, sent_predictions)

        mlflow.log_metric("sent_accuracy", sent_accuracy)
        mlflow.log_metric("lang_report", sent_report)

        

        

In [22]:
data_train("0")

/workspaces/mlops2023/venv/lib/python3.10/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/workspaces/mlops2023/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/workspaces/mlops2023/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/workspaces/mlops2023/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471:

MlflowException: Got invalid value               precision    recall  f1-score   support

                   0.00      0.00      0.00         4
          am       0.00      0.00      0.00         2
          ar       0.89      0.75      0.81       166
         art       0.00      0.00      0.00        13
          bn       1.00      0.75      0.86         4
          ca       0.98      0.65      0.78       187
         ckb       0.00      0.00      0.00         1
          cs       1.00      0.58      0.73        38
          cy       1.00      0.22      0.36        23
          da       0.67      0.27      0.38        30
          de       0.98      0.82      0.89       391
          el       0.75      0.82      0.78        11
          en       0.98      1.00      0.99    174195
          es       0.94      0.92      0.93      2292
          et       0.55      0.11      0.18       111
          eu       1.00      0.07      0.13        14
          fa       0.89      0.83      0.86   

In [ ]:
tweets_train, tweets_test, lang_labels_train, lang_labels_test, sent_labels_train, sent_labels_test = train_test_split(
            tweets, language_labels, sentiment_labels, test_size=0.2, random_state=42
        )

In [ ]:
tweets_train.head()

306497    lawindsor started gingrich amp contract americ...
620985    colorstv people still find reason hate her🙄🤦🏻‍...
292471                 p best always amp forever im serious
888059    yohfavourite boy friend poor think youre perio...
812986            ❗️ every time say end somewhere something
Name: Text, dtype: object

In [ ]:
lang_labels_train.head()

306497    en
620985    en
292471    en
888059    en
812986    en
Name: Language, dtype: object

In [ ]:
sent_labels_train.head()

306497      litigious
620985    uncertainty
292471       positive
888059       negative
812986    uncertainty
Name: Label, dtype: object

In [ ]:
sent_labels_train.unique()

array(['litigious', 'uncertainty', 'positive', 'negative'], dtype=object)

In [ ]:
# classificacao de language
language_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', LinearSVC())
])
# Train the language  classification model
language_pipeline.fit(tweets_train, lang_labels_train)

In [ ]:
from sklearn.naive_bayes import MultinomialNB

In [ ]:
# Define the pipeline for sentiment classification
sentiment_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', MultinomialNB())
])
# Train the sentiment classification model
sentiment_pipeline.fit(tweets_train, sent_labels_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [ ]:
# Predict the language of a tweet
tweet = "Predict the language of a tweet"
predicted_language = language_pipeline.predict([tweet])[0]
print("Predicted language:", predicted_language)

Predicted language: en


In [ ]:
# Predict the sentiment of a tweet
tweet = "reasonable material"
predicted_sentiment = sentiment_pipeline.predict([tweet])[0]
print("Predicted sentiment:", predicted_sentiment)

Predicted sentiment: litigious


In [ ]:
# Avaliar o modelo de identificação de idioma
lang_predictions = language_pipeline.predict(tweets_test)
lang_report = classification_report(lang_labels_test, lang_predictions)
print("Language identification report:\n", lang_report)

/workspaces/mlops2023/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/workspaces/mlops2023/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Language identification report:
               precision    recall  f1-score   support

                   0.00      0.00      0.00         4
          am       0.00      0.00      0.00         2
          ar       0.89      0.75      0.81       166
         art       0.00      0.00      0.00        13
          bn       1.00      0.75      0.86         4
          ca       0.98      0.65      0.78       187
         ckb       0.00      0.00      0.00         1
          cs       1.00      0.58      0.73        38
          cy       1.00      0.22      0.36        23
          da       0.67      0.27      0.38        30
          de       0.98      0.82      0.89       391
          el       0.75      0.82      0.78        11
          en       0.98      1.00      0.99    174195
          es       0.94      0.92      0.93      2292
          et       0.55      0.11      0.18       111
          eu       1.00      0.07      0.13        14
          fa       0.89      0.83      0.86     

/workspaces/mlops2023/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# Avaliar o modelo de identificação de sentimento
sent_predictions = sentiment_pipeline.predict(tweets_test)
sent_report = classification_report(sent_labels_test, sent_predictions)
print("Sentiment classification report:\n", sent_report)

Sentiment classification report:
               precision    recall  f1-score   support

   litigious       0.89      0.79      0.84     40875
    negative       0.74      0.90      0.81     52406
    positive       0.81      0.87      0.84     52602
 uncertainty       0.94      0.70      0.80     41686

    accuracy                           0.82    187569
   macro avg       0.84      0.81      0.82    187569
weighted avg       0.83      0.82      0.82    187569

